In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 26.7 MB/s eta 0:00:00


In [3]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import csv

In [4]:
mp_drawing = mp.solutions.drawing_utils#connection between points
mp_pose = mp.solutions.pose# pose algorthim
poses=mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [5]:


DATA_DIR = '/content/drive/MyDrive/naira'
data = []
label = []

mp_pose = mp.solutions.pose
poses = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []
        _x = []
        _y = []
        _z = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = poses.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for i in range(len(landmarks)):
                x = landmarks[i].x
                y = landmarks[i].y
                z = landmarks[i].z
                _x.append(x)
                _y.append(y)
                _z.append(z)
                data_aux.extend([x - min(_x), y - min(_y), z - min(_z)])

            data.append(data_aux)
            label.append(dir_)
        else:
            print(f"No pose landmarks detected in {os.path.join(DATA_DIR, dir_, img_path)}")

csv_file_path = 'data.csv'

# Writing data and labels to CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    header = ['Label'] + [f'Feature_{i}' for i in range(len(data[0]))]
    csv_writer.writerow(header)

    # Write data and labels
    for i in range(len(data)):
        row = [label[i]] + data[i]
        csv_writer.writerow(row)

print(f'Data with labels has been successfully written to {csv_file_path}')


No pose landmarks detected in /content/drive/MyDrive/naira/0/frame00258.png
No pose landmarks detected in /content/drive/MyDrive/naira/2/scene00135.png
No pose landmarks detected in /content/drive/MyDrive/naira/2/scene00189.png
Data with labels has been successfully written to data.csv


In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [8]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [9]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = 'data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
print(df)

      Label  Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0         0        0.0        0.0        0.0   0.017809        0.0   0.026307   
1         0        0.0        0.0        0.0   0.018244        0.0   0.026075   
2         0        0.0        0.0        0.0   0.014306        0.0   0.021840   
3         0        0.0        0.0        0.0   0.019168        0.0   0.019772   
4         0        0.0        0.0        0.0   0.018618        0.0   0.021767   
...     ...        ...        ...        ...        ...        ...        ...   
1216      2        0.0        0.0        0.0   0.029362        0.0   0.020556   
1217      2        0.0        0.0        0.0   0.029121        0.0   0.031320   
1218      2        0.0        0.0        0.0   0.028938        0.0   0.031210   
1219      2        0.0        0.0        0.0   0.028826        0.0   0.025740   
1220      2        0.0        0.0        0.0   0.028538        0.0   0.026162   

      Feature_6  Feature_7 

In [10]:
data = np.asarray(df.drop('Label', axis=1))
labels = np.asarray(df['Label'])
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [11]:
model=RandomForestClassifier()
model.fit(x_train,y_train)

RandomForestClassifier()

In [17]:
y_predict=model.predict(x_test)
print(y_predict)
accuracy=accuracy_score(y_predict,y_test)
print('{}% of sample predicted correctly'.format(accuracy*100))

[0 2 0 2 0 0 0 0 2 0 2 0 0 0 2 0 0 0 2 2 2 0 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 0 0 0 0 0 2 0 0 2 2 2 2 2 0 2 2 0 0 2 0 0 0 0 0 2 0 0 0 2 0 0 2 0 2 0 0 0
 0 0 0 0 0 0 0 0 2 0 2 2 0 2 2 0 0 2 2 2 0 0 0 2 0 2 0 0 0 2 0 0 0 0 0 0 0
 0 2 2 0 0 0 0 0 0 2 2 2 0 0 0 2 0 2 2 2 0 2 0 0 0 2 0 2 0 0 2 0 0 2 0 0 2
 0 2 0 0 0 0 0 2 0 0 2 0 2 0 2 0 0 0 0 0 0 0 2 2 2 0 0 0 2 0 0 2 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 0 2 2 0 2 0 0 2 0 0 2 2 0 2 0 0 0 0 0 2 0 0 0 2 0 2 0
 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 2 2 0 0 2 2 0 2]
100.0% of sample predicted correctly


In [13]:
import pickle
f = open('pose.h5', 'wb')
pickle.dump({'model': model}, f)
f.close()